# Question 1-Hashing


In [2]:
import math
import statistics
import numpy as np
from tqdm import tqdm

We decide to implement a particular hash function called ***DOT HASH FUNCTION***
Steps:
* 1. We fixed a large prime number , in our case $4294967311$ 
* 2. Divide each esadecimal number of the file ***hash.txt*** in 32 elements and put them into an array $x$
* 3. Generate 32 random number from 0 to the chosen prime number.
* 4. Calculate hash function as $h_a(x)=(\sum_{i=0}^{32} a_i x_i)mod p $

In [2]:
n=4294967311
coeff=np.random.randint(1,high=2**32-1,size=32,dtype='int64')
d={ 'a':10,'b':11,'c':12,'d':13,'e':14,'f':15}

def hash_function_2(s,n=n,coeff=coeff):
    out=0
    x=[]
    
    for elem in s:
        if elem in d:
            x.append(d[elem])
        else:
            x.append(int(elem))
    x=np.array(x)
    out=np.dot(x,coeff)
    
    return '{:032b}'.format(out%n)

The hash function defined above maps esadecimal integers to binary number with 32 digits.

# Hyper log log

Since we work with binary number of 32 bits the recommended number bits allocated to the buckets is 5,in this way we have $2^5=32$ different buckets. For each binary number we count the number of consecutive $0$ after the five bits reseved for the buckets.Then we store the max result **k** obtained in the corrispondent bucket.
Once we have processed all the inputs we calculate the number of unique values with the formula

$E=\alpha_m m^2(\sum_{j=1}^{m}2^{-M_j})^{-1}$  where $m$ is the number of buckets and $\alpha=0.697$ for 32 buckets.

The corrispondent error is $err$ =${1.04}\over{\sqrt(m)}$, so it is fixed for each number of buckets.

In [4]:
SIZE=32

def hyper_log_log_version(f):
    bucket=[0]*SIZE                            #32 bucket 
    bucket_index=0
    for i in tqdm(range(139000000)):
        riga=f.readline().strip('\n')
        a=hash_function_2(riga)
        bucket_index=0
        bucket_index= int(a[:5],2)
        
        cont_zero=0                           
        j=5 
        while a[j]== '0':
            cont_zero +=1
            j +=1
        
        if bucket[bucket_index]==0: 
            bucket[bucket_index]=cont_zero
        else :
            bucket[bucket_index]=max(bucket[bucket_index],cont_zero)
                
    s=0
    for elem in bucket:
        s+=2**(-elem)
    s=s**(-1)
    
    card=0.697*(SIZE**2)*s                    #cardinality
    err=1.04/math.sqrt(SIZE)                  #error
    
    return card,err

In [5]:
f=open('hash.txt','r', encoding='utf-8')

In [6]:
card,err=hyper_log_log_version2(f)

100%|█████████████████████████| 139000000/139000000 [50:28<00:00, 45895.85it/s]


In [7]:
f.close()

In [9]:
print(card)

101049525.91770463


In [10]:
print(err)

0.18384776310850234


# Universal Hash Function 

In [3]:
p=4294967311
a=np.random.randint(1,high=p,dtype='int64')
b=np.random.randint(0,high=p,dtype='int64')
n=2**32
def hash_function_2(esadecimal,a=a,b=b):
    x=int(esadecimal,16)
    out=((a*x+b)%p)%n
    return '{:032b}'.format(out)

In [6]:
card,err=hyper_log_log_version2(f)

100%|█████████████████████████| 139000000/139000000 [50:28<00:00, 45895.85it/s]


In [8]:
print(card)

60783598.077401005


# Question 3 - Sorting Algorithm

Given an array with **A** with n integer numbers, we define : 

> $ s = min{ A[1], ..., A[n] }$ -> the minimum value of the array, 

> $ b = max { A[1], ..., A[n] }$ -> the maximum value of the array,

> $ r= b-s$ -> the difference between max and min value

We create a second vector $B$ of length r+1 which contains the number of times that each value from s to b appears in $A$ .In this way the first element of $B$ represent the number of times that $s=min(A)$ appears ,consequently $B[i]$ represent the number of times that $i+min(A)$ appears in $A$. After visiting $A$ , starting from $B[0]$ we write on $A$ ,$B[i]$ copies of the value $i+min(A)$.

The complexity can be found looking ot the python code of this algorithm.

In [45]:
def sorting(A):
    print('Initial array                      ',A)
    s=min(A)
    b=max(A)
    r=b-s
    B=[0]*(r+1)
    
    for i in range(len(A)):
        B[A[i]-s]+=1
    print('Temporary array of the occurencies ',B)
    j=0
    for i in range(r+1):
        while B[i]!=0:
            A[j]=i+s
            B[i]-=1
            j+=1
    print('Ordered array                      ',A)

In [48]:
C=[4,2,8,10,3,3,5,6]

In [49]:
sorting(C)

Initial array                       [4, 2, 8, 10, 3, 3, 5, 6]
Temporary array of the occurencies  [1, 2, 1, 1, 1, 0, 1, 0, 1]
Ordered array                       [2, 3, 3, 4, 5, 6, 8, 10]


As we can see in the code we have to looking for the min and max ,this operations have a complexity of $O(n)$ while we have to visit all the array A.(Here we use buit-in function but in general we can calculate min and max visiting A only one time).

For creating B we need an other time n operations.

Then the last for look requires r+1 operations for overwriting A.

Leaving out constant operations such as {print,assignment,index incresement...},the final complexity is $O(n+r)$

